In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# api requests
# Senator api request
Senator_response = requests.get(
    'https://api.propublica.org/congress/v1/116/senate/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)
# House api request
House_response = requests.get(
    'https://api.propublica.org/congress/v1/116/house/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)

#convert responses into json
senate_json_response = Senator_response.json()
house_json_response = House_response.json()


In [2]:
# Senator array
Senate = senate_json_response['results'][0]["members"]
print(len(Senate))
# House array
House = house_json_response['results'][0]["members"]    
print(len(House))

102
450


In [3]:
#Congress Array
Congress = Senate + House
print(len(Congress))

552


In [4]:

# Convert array in Dataframe
df = pd.DataFrame(Congress)

In [5]:
df.keys()

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'date_of_birth', 'gender', 'party',
       'leadership_role', 'twitter_account', 'facebook_account',
       'youtube_account', 'govtrack_id', 'cspan_id', 'votesmart_id',
       'icpsr_id', 'crp_id', 'google_entity_id', 'fec_candidate_id', 'url',
       'rss_url', 'contact_form', 'in_office', 'cook_pvi', 'dw_nominate',
       'ideal_point', 'seniority', 'next_election', 'total_votes',
       'missed_votes', 'total_present', 'last_updated', 'ocd_id', 'office',
       'phone', 'fax', 'state', 'senate_class', 'state_rank', 'lis_id',
       'missed_votes_pct', 'votes_with_party_pct', 'votes_against_party_pct',
       'district', 'at_large', 'geoid'],
      dtype='object')

In [6]:
df = df.drop(["rss_url", "missed_votes_pct","votes_with_party_pct", "votes_against_party_pct", "cook_pvi","total_votes", "total_present","missed_votes", "at_large", 'date_of_birth', 'next_election', 'office'], axis=1)

In [7]:

df['image'] = "None"

In [8]:
# States for scraping
states = ["alabama", 'alaska',"arizona", "arkansas", "california", "colorado",'connecticut','delaware', 'florida','georgia', 'hawaii','idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri','montana', 'nebraska','nevada', 'new%20hampshire', 'new%20jersey','new%20mexico', 'new%20york', 'north%20carolina', 'north%20dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'rhode%20island', 'south%20carolina', 'south%20dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west%20virginia', 'wisconsin', 'wyoming' ]

Main_url = 'https://www.congress.gov/search?searchResultViewType=expanded&q={%22congress%22:[%22116%22],%22source%22:[%22members%22],%22search%22:%22'
back_of_Main_url = "%22}"
top_images_url = 'https://congress.gov'
#reps with no photos on Congress.gov
no_photos = []
Congress_dic = {}
Congress_dic['congress_info'] = [] 

In [9]:
# Checking to see if there is a picture on the website
def CheckForImg(str):
    try:
        rep_Img = rep.find("img")['src']
        return True
    except:
        return False

In [35]:
state_abrv = {"alabama":'AL', 'alaska':'AL',"arizona":'AZ', "arkansas":'AK', "california":'CA', "colorado":'CO','connecticut':'CT','delaware':'DE', 'florida':'FL','georgia':'GA', 'hawaii':'HI','idaho':'ID', 'illinois':'IL', 'indiana':'ID', 'iowa':'IA', 'kansas':'KS', 'kentucky':'KY', 'louisiana':'LA', 'maine':'ME', 'maryland':'MD', 'massachusetts':'MA', 'michigan':"MI", 'minnesota':'MN', 'mississippi':'MS', 'missouri':'MS','montana':'MT', 'nebraska':'NE','nevada':'NV', 'new%20hampshire':'NH', 'new%20jersey':'NJ','new%20mexico':'NM', 'new%20york':'NY', 'north%20carolina':'NC', 'north%20dakota':'ND', 'ohio':'OH', 'oklahoma':'OK', 'oregon':'OR', 'pennsylvania':'PA', 'rhode%20island':'RI', 'south%20carolina':'SC', 'south%20dakota':'SD', 'tennessee':'TN', 'texas':'TX', 'utah':'UT', 'vermont':'VT', 'virginia':'VA', 'washington':'WA', 'west%20virginia':'WV', 'wisconsin':'WI', 'wyoming':'WY'}

df.keys()

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'gender', 'party', 'leadership_role',
       'twitter_account', 'facebook_account', 'youtube_account', 'govtrack_id',
       'cspan_id', 'votesmart_id', 'icpsr_id', 'crp_id', 'google_entity_id',
       'fec_candidate_id', 'url', 'contact_form', 'in_office', 'dw_nominate',
       'ideal_point', 'seniority', 'last_updated', 'ocd_id', 'phone', 'fax',
       'state', 'senate_class', 'state_rank', 'lis_id', 'district', 'geoid',
       'image'],
      dtype='object')

In [41]:

# # for loop that is going thru every state
for state in states: 
    state_res = requests.get(Main_url+state+back_of_Main_url)
    state_soup = BeautifulSoup(state_res.text, 'html.parser')
    Rep_results = state_soup.findAll("li", {"class":"expanded"})

    for rep in Rep_results:
        photo = CheckForImg(rep)
        rep_name = rep.find("span", {"class":"result-heading"}).text.split(' ')[1:]
        rep_index = df.loc[(df['state'] == state_abrv[state]) & (df['last_name'] == rep_name[0][:-1])].index
        if photo:
            df.at[rep_index, 'image'] = rep_Img = rep.find("img")['src']

In [43]:
df[df['image'] == 'None']

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,gender,party,...,ocd_id,phone,fax,state,senate_class,state_rank,lis_id,district,geoid,image
21,C001113,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Catherine,None,Cortez Masto,None,F,D,...,ocd-division/country:us/state:nv,202-224-3542,None,NV,3,senior,S385,NaN,NaN,None
39,H001089,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Joshua,None,Hawley,None,M,R,...,ocd-division/country:us/state:mo,202-224-6154,None,MO,1,junior,S399,NaN,NaN,None
49,K000377,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Mark,None,Kelly,None,M,D,...,ocd-division/country:us/state:az,202-224-2235,None,AZ,3,,S406,NaN,NaN,None
95,V000128,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Chris,None,Van Hollen,None,M,D,...,ocd-division/country:us/state:md,202-224-4654,202-225-0375,MD,3,junior,S390,NaN,NaN,None
117,B001306,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Troy,None,Balderson,None,M,R,...,ocd-division/country:us/state:oh/cd:12,202-225-5355,None,OH,NaN,NaN,NaN,12,3912,None
120,B001300,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Nanette,None,Barragán,None,F,D,...,ocd-division/country:us/state:ca/cd:44,202-225-8220,None,CA,NaN,NaN,NaN,44,0644,None
132,B001303,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Lisa,None,Blunt Rochester,None,F,D,...,ocd-division/country:us/state:de/cd:1,202-225-4165,None,DE,NaN,NaN,NaN,At-Large,1000,None
153,C001097,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Tony,None,Cárdenas,None,M,D,...,ocd-division/country:us/state:ca/cd:29,202-225-6131,None,CA,NaN,NaN,NaN,29,0629,None
154,C001072,Representative,Rep.,https://api.propublica.org/congress/v1/members...,André,None,Carson,None,M,D,...,ocd-division/country:us/state:in/cd:7,202-225-4011,None,IN,NaN,NaN,NaN,7,1807,None
169,C001049,Representative,Rep.,https://api.propublica.org/congress/v1/members...,William,Lacy,Clay,None,M,D,...,ocd-division/country:us/state:mo/cd:1,202-225-2406,None,MO,NaN,NaN,NaN,1,2901,None


In [1]:
#exporting Dataframe to csv
df.to_csv('out.csv')


NameError: name 'df' is not defined